# PySparkのTips

細かいTips、テクニックをまとめる。

In [1]:
from glob import glob

import polars as pl
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import col
from pyspark.sql import functions as F
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml.functions import vector_to_array
import numpy as np

# Create a SparkSession。pythonからsparkを使う場合、セッションの作成が必要。
spark = SparkSession.builder.appName("Testing PySpark Example").getOrCreate()

# デフォルトのログレベルだと大量にログが出力されるので限定する。
spark.sparkContext.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/03 06:43:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# 各データ読み込み
df_receipt = spark.read.parquet("../../../100knocks-preprocess/docker/work/data/receipt.parquet")

# 店舗データ
df_store = spark.read.parquet("../../../100knocks-preprocess/docker/work/data/store.parquet")

# 顧客データ
df_customer = spark.read.parquet("../../../100knocks-preprocess/docker/work/data/customer.parquet")

# 製品データ
df_product = spark.read.parquet("../../../100knocks-preprocess/docker/work/data/product.parquet")

# 製品データ
df_category = spark.read.parquet("../../../100knocks-preprocess/docker/work/data/category.parquet")

# 特定カラムのユニークな値をリストとしてすべて取得
下記のような方法がある。どちらにせよめんどくさい。

In [4]:
# collectの結果の各値は対象カラムをキーとする辞書のような形で取得できる
[v["gender_cd"] for v in df_customer.select("gender_cd").distinct().collect()]

['0', '9', '1']

In [5]:
df_customer.select("gender_cd").dropDuplicates().rdd.flatMap(lambda x: x).collect()

['0', '9', '1']

In [6]:
# Dataframeとして取得したいなら.distinctでOK
df_customer.select("gender_cd").distinct().show()

+---------+
|gender_cd|
+---------+
|        0|
|        9|
|        1|
+---------+



# joinの結合条件にcontainsを用いる
結構便利。left joinで左のカラムに右のカラムの値が含まれている行に対して結合したい場合などに使える。

In [29]:
# サンプルデータの作成
data_a = [
    (1, "This is a sample message"),
    (2, "Another example message"),
    (3, "Message with a keyword"),
    (4, "No match here"),
    (5, "samplesample"),
    (6, "sample keyword"),
]
columns_a = ["id", "message"]
df_a = spark.createDataFrame(data_a, columns_a)

data_b = [
    (1, "sample", "MSG001"),
    (2, "example", "MSG002"),
    (3, "keyword", "MSG003")
]
columns_b = ["id" ,"message_key", "MSG_No"]
df_b = spark.createDataFrame(data_b, columns_b)


In [30]:
df_a.show()

+---+--------------------+
| id|             message|
+---+--------------------+
|  1|This is a sample ...|
|  2|Another example m...|
|  3|Message with a ke...|
|  4|       No match here|
|  5|        samplesample|
|  6|      sample keyword|
+---+--------------------+



In [31]:
df_b.show()

+---+-----------+------+
| id|message_key|MSG_No|
+---+-----------+------+
|  1|     sample|MSG001|
|  2|    example|MSG002|
|  3|    keyword|MSG003|
+---+-----------+------+



In [32]:
df_a.join(
    df_b,
    # 結合条件にcontainsを使用。messageにmessage_keyが含まれていれば紐づける
    F.contains(df_a.message, df_b.message_key),
    "left"
).show()

+---+--------------------+----+-----------+------+
| id|             message|  id|message_key|MSG_No|
+---+--------------------+----+-----------+------+
|  1|This is a sample ...|   1|     sample|MSG001|
|  2|Another example m...|   2|    example|MSG002|
|  3|Message with a ke...|   3|    keyword|MSG003|
|  4|       No match here|NULL|       NULL|  NULL|
|  5|        samplesample|   1|     sample|MSG001|
|  6|      sample keyword|   1|     sample|MSG001|
|  6|      sample keyword|   3|    keyword|MSG003|
+---+--------------------+----+-----------+------+



"sample keyword"のように２つの単語に紐づく場合は2行紐づく。

# Dataframeの任意の行番号範囲を取り出す

In [ ]:
dataset = df_receipt.withColumn(
    "sales_month",
    F.col("sales_ymd").substr(0, 6) # yyyyMMdd -> yyyyMM形式にする
).groupBy("sales_month").agg(   # 月次ごとに集計
    F.sum("amount").alias("total_amount")
).withColumn(
    "row_num",
    F.row_number().over(Window.orderBy("sales_month")) - 1
)

dataset.show(18)

+-----------+------------+-------+
|sales_month|total_amount|row_num|
+-----------+------------+-------+
|     201701|      902056|      0|
|     201702|      764413|      1|
|     201703|      962945|      2|
|     201704|      847566|      3|
|     201705|      884010|      4|
|     201706|      894242|      5|
|     201707|      959205|      6|
|     201708|      954836|      7|
|     201709|      902037|      8|
|     201710|      905739|      9|
|     201711|      932157|     10|
|     201712|      939654|     11|
|     201801|      944509|     12|
|     201802|      864128|     13|
|     201803|      946588|     14|
|     201804|      937099|     15|
|     201805|     1004438|     16|
|     201806|     1012329|     17|
+-----------+------------+-------+
only showing top 18 rows



※PySparkのDataframeの部分行だけ取り出すには上記のようWindow関数を使って行番号を振るしかなさそう。  
  メモリに読み込まずに先頭から指定したレコード数だけ取り出す場合はlimitが使えるが、どこからどこまで取り出すという指定はできない。

In [ ]:
train_size = 12
val_size = 6
offset = 6 # 次のtrainをどこから始めるか

In [ ]:
train_data = []
val_data = []

# 12か月ごとに学習データ、6か月ごとに検証データを定義する
for i in range(3):
    train_start = offset * i
    train_end = train_start + train_size - 1
    val_start = train_end + 1
    val_end = val_start + offset - 1
    train_data.append(dataset.filter(F.col("row_num").between(train_start, train_end)))
    val_data.append(dataset.filter(F.col("row_num").between(val_start, val_end)))

# pyspark.sql.functions.coalesceの挙動
名前から何をするのかわかりづらいので、挙動をおさらいしておく。  
coalesceは”合体する”といった意味の英単語で、引数を左端から調べて最初に現れた非NULL値を返す関数である。  
PySparkだけでなくSQLの文法にもある。  

[pyspark.sql.functions.coalesceのドキュメント](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.coalesce.html?highlight=coale#pyspark.sql.functions.coalesce)

In [14]:
df = spark.createDataFrame([
    (None, None, None),
    (1, None, None),
    (None, 2, None),
    (None, None, 3),
    (None, 4, 5),
], schema='a long, b long, c long')
df.show()

+----+----+----+
|   a|   b|   c|
+----+----+----+
|NULL|NULL|NULL|
|   1|NULL|NULL|
|NULL|   2|NULL|
|NULL|NULL|   3|
|NULL|   4|   5|
+----+----+----+



In [17]:
# 2つのカラムをcoalesceする
df.select(F.coalesce("a", "b")).show()

+--------------+
|coalesce(a, b)|
+--------------+
|          NULL|
|             1|
|             2|
|          NULL|
|             4|
+--------------+



In [16]:
# 3つのカラムをcoalesceする
df.select(F.coalesce("a", "b", "c")).show()

+-----------------+
|coalesce(a, b, c)|
+-----------------+
|             NULL|
|                1|
|                2|
|                3|
|                4|
+-----------------+



上記のように各行を左のカラムから右のカラムへ（a->b）調べていき、  
最初にヒットしたNullでない値を採用して一つのカラムを返す。  
全カラムNullの場合は単にNullになる。

# Nullの置換

## pyspark.sql.DataFrame.fillnaによる置換
王道

In [18]:
df.show()

+----+----+----+
|   a|   b|   c|
+----+----+----+
|NULL|NULL|NULL|
|   1|NULL|NULL|
|NULL|   2|NULL|
|NULL|NULL|   3|
|NULL|   4|   5|
+----+----+----+



In [23]:
# 複数列のNullを置換
df.fillna({"a": 0, "b": 999}).show()

+---+---+----+
|  a|  b|   c|
+---+---+----+
|  0|999|NULL|
|  1|999|NULL|
|  0|  2|NULL|
|  0|999|   3|
|  0|  4|   5|
+---+---+----+



## coalesceによる置換

In [24]:
df.show()

+----+----+----+
|   a|   b|   c|
+----+----+----+
|NULL|NULL|NULL|
|   1|NULL|NULL|
|NULL|   2|NULL|
|NULL|NULL|   3|
|NULL|   4|   5|
+----+----+----+



In [34]:
expr = {}
for col in df.columns:
    expr[col] = F.coalesce(col, F.lit(0))

In [35]:
df.withColumns(
    expr
).show()

+---+---+---+
|  a|  b|  c|
+---+---+---+
|  0|  0|  0|
|  1|  0|  0|
|  0|  2|  0|
|  0|  0|  3|
|  0|  4|  5|
+---+---+---+



24/08/03 06:17:28 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:123)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:688)
	at org.apache.spark.storage.BlockManagerMasterE

0の定数カラムとcoalesceするところがポイント。  
複数行にやろうとすると面倒。  
他にもF.whenを使用する方法があるが、当然なので割愛。

# FULL OUTER JOIN(外部結合の注意点)
PySparkに限った話ではないが、JOIN後に残すカラムは慎重に選ぶべき。  
例えば、テーブルAの主キーカラムAとテーブルBの主キーカラムBについて、FULL JOINで  
カラムA==カラムBを条件としてJoinする場合、  
Join後に残す主キーをカラムAだけとした場合、カラムBにしかなかった値はNULLになってしまう。  
カラムBにあった値もJOIN後の主キーに残すためには追加の処理が必要。  
JOINの仕組みを考えれば当たり前だが、注意が必要。

In [2]:
# テーブルAのサンプルデータの作成
data_a = [
    (1, "Alice"),
    (2, "Bob"),
    (3, "Charlie")
]
columns_a = ["id_a", "name_a"]
df_a = spark.createDataFrame(data_a, columns_a)

# テーブルBのサンプルデータの作成
data_b = [
    (3, "David"),
    (4, "Eve"),
    (5, "Frank")
]
columns_b = ["id_b", "name_b"]
df_b = spark.createDataFrame(data_b, columns_b)

In [4]:
# 残すカラムを指定しない場合
df_a.join(
    df_b, 
    df_a.id_a == df_b.id_b, 
    "full_outer"
).show()

+----+-------+----+------+
|id_a| name_a|id_b|name_b|
+----+-------+----+------+
|   1|  Alice|NULL|  NULL|
|   2|    Bob|NULL|  NULL|
|   3|Charlie|   3| David|
|NULL|   NULL|   4|   Eve|
|NULL|   NULL|   5| Frank|
+----+-------+----+------+



In [5]:
# 主キーとしてid_aだけ残す
df_a.join(
    df_b, 
    df_a.id_a == df_b.id_b, 
    "full_outer"
).select(
    "id_a", "name_a", "name_b"
).show()

+----+-------+------+
|id_a| name_a|name_b|
+----+-------+------+
|   1|  Alice|  NULL|
|   2|    Bob|  NULL|
|   3|Charlie| David|
|NULL|   NULL|   Eve|
|NULL|   NULL| Frank|
+----+-------+------+



思いとして、id_bに相当するname_bはあるので、id_aにid_bの値も残っててくれたらありがたいが、  
当然そうはならない。対処としては下記のようにid_a, id_bをcoalesceに指定して、NULLを埋めて結合する方法と、  
whenを使用する方法がある。

In [7]:
# coalesceを使う方法
df_a.join(
    df_b, 
    df_a.id_a == df_b.id_b, 
    "full_outer"
).withColumn(
    "id",
    F.coalesce("id_a", "id_b")
).select(
    "id", "name_a", "name_b"
).show()

+---+-------+------+
| id| name_a|name_b|
+---+-------+------+
|  1|  Alice|  NULL|
|  2|    Bob|  NULL|
|  3|Charlie| David|
|  4|   NULL|   Eve|
|  5|   NULL| Frank|
+---+-------+------+



In [8]:
# whenを使う方法
df_a.join(
    df_b, 
    df_a.id_a == df_b.id_b, 
    "full_outer"
).withColumn(
    "id",
    F.when(F.col("id_a").isNull(), F.col("id_b")).otherwise(F.col("id_a"))
).select(
    "id", "name_a", "name_b"
).show()

+---+-------+------+
| id| name_a|name_b|
+---+-------+------+
|  1|  Alice|  NULL|
|  2|    Bob|  NULL|
|  3|Charlie| David|
|  4|   NULL|   Eve|
|  5|   NULL| Frank|
+---+-------+------+

